In [41]:
from sys import stdout
 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
 
from scipy.signal import savgol_filter
 
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale 
from numpy.polynomial.polynomial import Polynomial

from keras.models import Sequential
from keras.layers import Dense

from progressbar import ProgressBar

from sklearn.preprocessing import StandardScaler

In [42]:

#import training datadet
#import test data
inter = pd.ExcelFile('own LC data_v2.xlsx')
inter = pd.read_excel(inter,'Comb1')
#print(inter.info())
comb = inter.dropna(axis=0)
#print(comb.info())
comb = comb[comb["tR (min)"] != "?"]
print(comb.shape[0])
print(comb.shape[1])
col_to_remove=[]
#preprocess
for col in comb.columns[1:]:
    if (comb[col] == 0).all():
        comb = comb.drop([col], axis = 1)
        col_to_remove.append(col)
comb = comb.reset_index(drop=True)
X_CAL = comb.drop(['Project','Column','Gnum','tR (min)'], axis=1)
print(X_CAL.info())
#print(X_CAL.iloc[:,0:12])
y = {'tR (min)': [round(x,4) for x in comb['tR (min)']]}
y = pd.DataFrame(comb, columns = ['tR (min)'])


inter1 = pd.ExcelFile('data for LSR2023Feb.xlsx')
inter1 = pd.read_excel(inter1,'Sheet2')
#print(inter.info())
test = inter1.dropna(axis=0)
#print(comb.info())
test = test[test["tR (min)"] != "?"]
#preprocess
test = test.drop(col_to_remove,axis=1)
X_test = test.drop(['Project','Column','Gnum','tR (min)'], axis=1)
print(X_test.info())
#print(X_test.iloc[:,0:12])
y_test = {'tR (min)': [round(x,4) for x in test['tR (min)']]}
y_test = pd.DataFrame(test, columns = ['tR (min)'])
print(y_test)

1790
185
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Columns: 141 entries, H to fsulfone
dtypes: float64(100), int64(41)
memory usage: 1.9 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 160 entries, 0 to 221
Columns: 141 entries, H to fsulfone
dtypes: float64(100), int64(41)
memory usage: 177.5 KB
None
     tR (min)
0       6.088
1       5.550
2       5.488
4       6.959
5       6.791
..        ...
217     8.299
218     8.246
219     6.642
220     7.270
221     6.711

[160 rows x 1 columns]


In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def HPLC_COLUMN_TRANSFER_SEARCH(X_CAL, Y_CAL, ind_table, COL_A, COL_B):
    temp_A = ind_table[ind_table['Column'] == COL_A].index
    temp_B = ind_table[ind_table['Column'] == COL_B].index

    Gnum_A = ind_table.loc[temp_A, 'Gnum'].values
    Gnum_B = ind_table.loc[temp_B, 'Gnum'].values
    
    X_CAL_A = X_CAL[temp_A, :]
    Y_CAL_A = Y_CAL[temp_A, :]

    X_CAL_B = X_CAL[temp_B, :]
    Y_CAL_B = Y_CAL[temp_B, :]

    UNIQUE_LIST = np.unique(np.concatenate((Gnum_A, Gnum_B)))


    X_SEL_A = np.empty((0, X_CAL.shape[1]))
    Y_SEL_A = np.empty((0, Y_CAL.shape[1]))
    X_SEL_B = np.empty((0, X_CAL.shape[1]))
    Y_SEL_B = np.empty((0, Y_CAL.shape[1]))
    Gnum = np.empty((0,), dtype=str)
    
    for unique_item in UNIQUE_LIST:

        ind_A = np.where(Gnum_A == unique_item)[0]
        ind_B = np.where(Gnum_B == unique_item)[0]

        if ind_A.size == 0 or ind_B.size == 0:
            continue
        elif ind_A.size == ind_B.size:
            for k in range(len(ind_A)):
                # column 11 is the pH
                temp = X_CAL_A[ind_A[k], 11] - X_CAL_B[ind_B, 11]

                ind = np.where(temp == 0)
                ind = np.asarray(ind).astype('int16')

                if ind.size != 0:
                    X_SEL_A = np.vstack([X_SEL_A, X_CAL_A[ind_A[k], :]])
                    Y_SEL_A = np.vstack([Y_SEL_A, Y_CAL_A[ind_A[k], :]])
                    X_SEL_B = np.vstack([X_SEL_B, np.mean(X_CAL_B[ind_B[ind], :], axis=0)])
                    Y_SEL_B = np.vstack([Y_SEL_B, np.mean(Y_CAL_B[ind_B[ind], :], axis=0)])
                    Gnum = np.append(Gnum, unique_item)
        elif ind_A.size < ind_B.size:
            for k in range(len(ind_A)):
                temp = X_CAL_A[ind_A[k], 11] - X_CAL_B[ind_B, 11]

                ind = np.where(temp == 0)
                ind = np.asarray(ind).astype('int16')

                if ind.size != 0:
                    X_SEL_A = np.vstack([X_SEL_A, X_CAL_A[ind_A[k], :]])
                    Y_SEL_A = np.vstack([Y_SEL_A, Y_CAL_A[ind_A[k], :]])
                    X_SEL_B = np.vstack([X_SEL_B, np.mean(X_CAL_B[ind_B[ind], :], axis=0)])
                    Y_SEL_B = np.vstack([Y_SEL_B, np.mean(Y_CAL_B[ind_B[ind], :], axis=0)])
                    Gnum = np.append(Gnum, unique_item)
        elif ind_A.size > ind_B.size:
            for k in range(len(ind_B)):
                temp = X_CAL_A[ind_A, 11] - X_CAL_B[ind_B[k], 11]

                ind = np.where(temp == 0)
                ind = np.asarray(ind).astype('int16')

                if ind.size != 0:
                    X_SEL_A = np.vstack([X_SEL_A, np.mean(X_CAL_A[ind_A[ind], :], axis=0)])
                    Y_SEL_A = np.vstack([Y_SEL_A, np.mean(Y_CAL_A[ind_A[ind], :], axis=0)])
                    X_SEL_B = np.vstack([X_SEL_B, X_CAL_B[ind_B[k], :]])
                    Y_SEL_B = np.vstack([Y_SEL_B, Y_CAL_B[ind_B[k], :]])
                    Gnum = np.append(Gnum, unique_item)
    
    RESULTS = {
        'X_SEL_A': X_SEL_A,
        'X_SEL_B': X_SEL_B,
        'Y_SEL_A': Y_SEL_A,
        'Y_SEL_B': Y_SEL_B,
        'Gnum': Gnum
    }

    if len(Y_SEL_A) == len(Y_SEL_B):
        plt.plot(Y_SEL_A, Y_SEL_B, 'ro')
        plt.xlabel(f'measured Rt on {COL_A} (min)')
        plt.ylabel(f'measured Rt on {COL_B} (min)')
        plt.show()
    
    return RESULTS

In [92]:

def HPLC_COLUMN_TRANSFER_LSR2023Feb_SEARCH(X_CAL,Y_CAL,X_test,Y_test,Project_IN):
    
    UNIQUE_LIST = test['Column'].unique()
    print(UNIQUE_LIST)
    print(test['Project'].unique())
    RMSEP_1 = []
    RMSEP_2 = []
    LABEL = []
    Project = Project_IN  # Example project

    for i in range(5):
        for k in range(i + 1, 5):
            ind_cal = (test['Project'] == Project) & (test['Column'] == UNIQUE_LIST[i])
            ind_val = (test['Project'] == Project) & (test['Column'] == UNIQUE_LIST[k])
            #print(ind_cal.sum())
            #print(ind_val.sum())
            if ind_cal.sum() < 3 or ind_val.sum() < 3:
                continue
            else: 
                #print('Pete')
                # ANN
                XScaler=StandardScaler()
                YScaler=StandardScaler()
                temp_x=np.vstack([X_CAL, X_test.loc[ind_cal,:]])
                temp_y=np.vstack([y, Y_test.loc[ind_cal,:]])
                temp_x_test = X_test.loc[ind_val,:]
                temp_y = np.asarray(temp_y).astype('float32')
                temp_x = np.asarray(temp_x).astype('float32')
                temp_x_test = np.asarray(temp_x_test).astype('float32')
    
                X_scaled=XScaler.fit_transform(temp_x)
                y_scaled=YScaler.fit_transform(temp_y)
                X_test_scaled=XScaler.transform(temp_x_test)

                # create ANN model
                model = Sequential()
                # Defining the Input layer and FIRST hidden layer, both are same!
                model.add(Dense(units=5, input_dim=141, kernel_initializer='normal', activation='relu'))
                # Defining the Second layer of the model
                # after the first layer we don't have to specify input_dim as keras configure it automatically
                model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
                # Defining the Third layer of the model
                model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
                # The output neuron is a single fully connected node 
                # Since we will be predicting a single number
                model.add(Dense(1, kernel_initializer='normal'))
                # Compiling the model
                model.compile(loss='mean_squared_error', optimizer='adam')    
                # Fitting the ANN to the Training set
                model.fit(X_scaled, y_scaled ,batch_size = 50, epochs = 50, verbose=0)

                #X_test = X_test.astype('float32')
                # Generating Predictions on testing data
                Predictions=model.predict(X_test_scaled)
                # Scaling the predicted Price data back to original price scale
                Predictions=YScaler.inverse_transform(Predictions)
                RMSEP_temp = np.sqrt(mean_squared_error(Y_test.loc[ind_val], Predictions))
                tempA=test.loc[ind_cal, 'Gnum'].values
                tempA = [[item] for item in tempA]
                tempB=test.loc[ind_val, 'Gnum'].values
                tempB = [[item] for item in tempB]
                
                LABEL.append([
                    UNIQUE_LIST[i],
                    tempA,
                    UNIQUE_LIST[k],
                    tempB,
                    ind_val.sum(),
                    Y_test.loc[ind_val], 
                    Predictions
                ])

                RMSEP_2.append(RMSEP_temp)

               

    OUTPUT = {
        'LABEL': LABEL,
        'RMSEP_2': RMSEP_2
    }
    
    return OUTPUT

In [109]:
OUTPUT=HPLC_COLUMN_TRANSFER_LSR2023Feb_SEARCH(X_CAL,y,X_test,y_test,'Nav1.7')

['Zorbax Extend C18' 'Kinetix XB-C18' 'Poroshell EC-C18' 'Cortecs C18+'
 'ACE C18']
['Nav1.7' 'LepB' 'HPK1' 'iJAK' 'IRE1' 'Jak1' 'KRAS' 'NLRP3' 'RIP1K' 'TEAD'
 'TRPA1' 'Vps34']
1/1 [==============================] - 0s 63ms/step


In [110]:
#print(OUTPUT['LABEL'][0][1])

samp=[]
columnA=[]
columnB=[]
GnumA=np.array
GnumB=np.array
Rt_mea=np.array
Rt_pre=np.array
rmse_ANN=[]

for i in range(len(OUTPUT['LABEL'])):
    columnA.append(OUTPUT['LABEL'][i][0])
    columnB.append(OUTPUT['LABEL'][i][2])
    tempA=np.array(OUTPUT['LABEL'][i][1])
    GnumA=np.vstack([GnumA,tempA])
    tempB=np.array(OUTPUT['LABEL'][i][3])
    GnumB=np.vstack([GnumB,tempB])
    samp.append(OUTPUT['LABEL'][i][4])
    temp_mea=np.array(OUTPUT['LABEL'][i][5])
    Rt_mea=np.vstack([Rt_mea,temp_mea])
    temp_pre=np.array(OUTPUT['LABEL'][i][6])
    Rt_pre=np.vstack([Rt_pre,temp_pre])
    rmse_ANN.append(OUTPUT['RMSEP_2'][i])



#tempcolA=np.array(columnA)
#columnA=tempcolA.reshape(-1,1)
#tempcolB=np.array(columnB)
#columnB=tempcolB.reshape(-1,1)
#print(GnumA)
tempA=np.array(GnumA)
GnumA=tempA.reshape(-1,1)
#print(GnumA)
tempB=np.array(GnumB)
GnumB=tempB.reshape(-1,1)

columnA=pd.DataFrame(columnA,columns=['columnA'])
columnB=pd.DataFrame(columnB,columns=['columnB'])
GnumA=pd.DataFrame(GnumA[1:,:],columns=['GnumA'])
GnumB=pd.DataFrame(GnumB[1:,:],columns=['GnumB'])
samp=pd.DataFrame(samp,columns=['# of samples'])
Rt_mea=pd.DataFrame(Rt_mea[1:,:],columns=['measured Rt'])
Rt_pre=pd.DataFrame(Rt_pre[1:,:],columns=['predicted Rt'])
rmse_ANN=pd.DataFrame(rmse_ANN,columns=['rmse_ANN'])

results = pd.concat([columnA, columnB, GnumA, GnumB, samp, Rt_mea, Rt_pre, rmse_ANN],axis=1)

results.to_csv('COLUMN_TRANSFER_LSR2023Feb.csv')